**{DEV}\[TODO\]**
- everything

---
### Chapter 1: System Modelling
---
# **Tutorial 1.d: Pendulum in 3D**
**Aim:**

### **Contents**:
* [3D Modelling](#3D-Modelling)
* [Spherical Pendulum](#Spherical-Pendulum)
* [Hinge Pendulum](#Hinge-Pendulum)

In [ ]:
mass    = 1.0
length  = 0.5
dCOM    = 0.5

I_psi_val = mass*length**2/12
I_th_val  = mass*length**2/12

r = 0.01 # an example value of the width of the link
I_phi_val = mass*r**2/2

\***NOTE:** strictly speaking, $I_{\psi} = I_{\theta} = \frac{1}{12}*m(L^{2}+3r^{2})$, but r is small, so we don't bother with it.

## **3D Modelling**

In [ ]:
# import libraries
import sympy as sym
import numpy as np
import time # for benchmarking different models

import multiprocessing as mp #multiprocessing, just for the simplification steps

from IPython.display import display, HTML #for pretty printing
display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 32em; }</style>"))

# define time-logging function for benchmarking
def log_time(t_start=None, msg=""):
    t_end = time.perf_counter()
    if t_start == None:
        if msg == "": print("Timer initialized")
        else: print(msg)
    else: print(msg + f": {t_end - t_start:.6f} sec")
    return t_end

def mp_trigsimp(m):
    mp_data = [(m[r,c]) for r in range(m.rows) for c in range(m.cols)]
    if __name__ == '__main__':
        p = mp.Pool()
        results = p.map(sym.trigsimp, mp_data)
        p.close()
        p.join()
    return sym.Matrix(results).reshape(m.rows,m.cols)

# create symbolic variables - Generalized for n-link
nlinks = 2
links = np.arange(nlinks)

# --------------------------------------------------------------------------------------------------------------

# system parameters
g = sym.symbols('g')

m  = [sym.symbols( 'm_{%s}' %(i+1)) for i in links] # mass of links
l  = [sym.symbols( 'l_{%s}' %(i+1)) for i in links] # length of links
d  = [sym.symbols( 'd_{%s}' %(i+1)) for i in links] # distance to COM of links (from origin)

I_phi = [sym.symbols('I_{\\phi_{%s}}'   %(i+1)) for i in links] # moment of intertia of links
I_th  = [sym.symbols('I_{\\theta_{%s}}' %(i+1)) for i in links]
I_psi = [sym.symbols('I_{\\psi_{%s}}'   %(i+1)) for i in links]

# generalized coordinates
X0, Y0, Z0 = sym.symbols(['X_{0}','Y_{0}', 'Z_{0}']) # fixed position of first link

phi   = [sym.symbols(       '\\phi_{%s}' %(i+1)) for i in links] #positions
dphi  = [sym.symbols( '\dot{\\phi}_{%s}' %(i+1)) for i in links] #velocities
ddphi = [sym.symbols('\ddot{\\phi}_{%s}' %(i+1)) for i in links] #accelerations

th   = [sym.symbols(       '\\theta_{%s}' %(i+1)) for i in links] #positions
dth  = [sym.symbols( '\dot{\\theta}_{%s}' %(i+1)) for i in links] #velocities
ddth = [sym.symbols('\ddot{\\theta}_{%s}' %(i+1)) for i in links] #accelerations

psi   = [sym.symbols(       '\\psi_{%s}' %(i+1)) for i in links] #positions
dpsi  = [sym.symbols( '\dot{\\psi}_{%s}' %(i+1)) for i in links] #velocities
ddpsi = [sym.symbols('\ddot{\\psi}_{%s}' %(i+1)) for i in links] #accelerations

q   = sym.Matrix([[  phi[i],  th[i],  psi[i]] for i in links]) # fixed base so only rotations
dq  = sym.Matrix([[ dphi[i], dth[i], dpsi[i]] for i in links])
ddq = sym.Matrix([[ddphi[i],ddth[i],ddpsi[i]] for i in links])

q   =   q.reshape(q.rows * q.cols, 1)
dq  =  dq.reshape(q.rows * q.cols, 1)
ddq = ddq.reshape(q.rows * q.cols, 1)

# display(q)

In [ ]:
# STEP 1 & 2

order = ['z','y','x'] # rotation order

# helper function
# the 3D system space coordinates are [x;y;z;psi;th;phi].
# It is easier to split up the translations and orientations into separate variables in 3D
def get_RE(phi, th, psi, order): # generate rotation matrix
    o1, o2, o3 = order
    
    # Define rotation matrices for each axis of rotation
    Rx = sym.Matrix([[1,            0,             0],
                     [0, sym.cos(phi), -sym.sin(phi)],
                     [0, sym.sin(phi),  sym.cos(phi)]])

    Ry = sym.Matrix([[ sym.cos(th), 0, sym.sin(th)],
                     [           0, 1,           0],
                     [-sym.sin(th), 0, sym.cos(th)]])

    Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                     [sym.sin(psi),  sym.cos(psi), 0],
                     [           0,             0, 1]])
    
    # map the rotation matrices to their axes
    Rdict = {'x': Rx,
             'y': Ry,
             'z': Rz}
    
    # Define rotation matrices for moving between the body frame and each of the previous frames
    R32 = Rdict[o3]     # undo last 1 rotation
    R31 = Rdict[o2]*R32 # undo last 2 rotations
    R30 = Rdict[o1]*R31 # undo  all 3 rotations
    
    # Euler Angle Rates
    
    # Define axis vectors in their own frames
    axes = {'x': sym.Matrix([[1],[0],[0]]),
            'y': sym.Matrix([[0],[1],[0]]),
            'z': sym.Matrix([[0],[0],[1]])}
    
    # Transform these axes so that they are all in the BODY frame *not inertial
    E_cols = {o3: R32.transpose()*axes[o3],
              o2: R31.transpose()*axes[o2],
              o1: R30.transpose()*axes[o1]}
    
    # Rearrange axes into the order x,y,z (so that this matrix can multiply by vectors of the form x,y,z)
    E = sym.Matrix.hstack(E_cols['x'], E_cols['y'], E_cols['z']) #hstack -> horizontally stack
    # E = sym.Matrix([E_cols['x'], E_cols['y'], E_cols['z']]).reshape(3,3).transpose() # does the same thing as above
    
    return [R30, E] # return 1) the rotation from body to inertial, and 2) the euler rates matrix

# positions of each link in their own reference frames
rCOMs_0 = []
rEnds_0 = [sym.Matrix([[X0],[Y0],[Z0]])] # positions of the tops/bottoms of links
drCOMs  = []
omega   = []

R, E = [0]*nlinks, [0]*nlinks

for i in range(nlinks): # enumerate returns the index and the value at that index
    R[i], E[i] = get_RE(phi[i], th[i], psi[i], order)
    
    rLen_n = sym.Matrix([[0],[0],[-l[i]]])
    rTop_0 = rEnds_0[i]

    rCOMs_0.append( R[i]*rLen_n*d[i] + rTop_0)
    rEnds_0.append( R[i]*rLen_n      + rTop_0) # indexing: top of link matches COM in rCOM_0. extra row in rEnds_0 = bottom of nth link
    
    drCOM = rCOMs_0[i].jacobian(q)*dq
    drCOMs.append(sym.trigsimp(drCOM))

    omega.append( E[i]*sym.Matrix(dq[3*i : 3*(i+1)]) )

In [ ]:
# STEP 3
T = sym.Matrix([[0]])
V = sym.Matrix([[0]])

for i in range(nlinks):
    T += 0.5*m[i]*drCOMs[i].T*drCOMs[i]
    T += sym.Matrix([0.5*I_phi[i]*omega[i][0]**2 + 0.5*I_th[i]*omega[i][1]**2 + 0.5*I_psi[i]*omega[i][2]**2])
    
    V += sym.Matrix([m[i]*g*rCOMs_0[i][2]])

# T = mp_trigsimp(T)

# STEP 4
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial
    
# term 3
Lg3 = sym.Matrix([T]).jacobian(q) # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q) # partial of V in q


# STEP 5
tau  = [sym.symbols( 'tau_{%s}'%s) for s in q]
tau = sym.Matrix(tau)
tau = tau.reshape(nlinks,3)
# display(tau)
Q = sym.zeros(1,len(q))
for i in range(nlinks):
    torque = tau.row(i)
    if i == 0:
        dW = torque.dot(omega[i])
    else:
        dW = torque.dot(omega[i] - omega[i-1])
    Q += sym.Matrix([dW]).jacobian(dq)
# display(Q.T)
# STEP 6
EOM = Lg1 - Lg3 + Lg4 - Q
EOM = EOM.T

# EOM = mp_trigsimp(EOM)
# display(EOM)

In [ ]:
N = 20
h = 0.01
t = np.arange(0,N*h,h)

#parameters
X0val, Y0val, Z0val,  = 0, 0, 2

mass    = 1.0
length  = 0.5
dCOM    = 0.5

I_phi_val = mass*length**2/12
I_th_val  = mass*length**2/12

r = 0.01 # an example value of the width of the link
I_psi_val = mass*r**2/2

tau_vals = [3*np.sin(8*np.pi*t), 3*np.cos(8*np.pi*t), 0*t]
# --------------------------------------------------------------------------------

parameter_values = [(X0,X0val),(Y0,Y0val),(Z0,Z0val),
                    (g,9.81)]+\
                    [(m[i],mass) for i in range(nlinks)]+\
                    [(l[i],length) for i in range(nlinks)]+\
                    [(d[i],dCOM) for i in range(nlinks)]+\
                    [(I_psi[i],I_psi_val) for i in range(nlinks)]+\
                    [(I_th[i],I_th_val) for i in range(nlinks)]+\
                    [(I_phi[i],I_phi_val) for i in range(nlinks)]

# substitute parameters into EOM
EOM_sub_params = EOM.subs(parameter_values)

# initial conditions
q_vals = np.zeros((3*nlinks, N+1))
dq_vals = np.zeros((3*nlinks, N+1))

rEnds_v2 = sym.Matrix(rEnds_0)
r_sub_params = rEnds_v2.subs(parameter_values)
r_sub = []

for n in range(1,N+1):
    # conditions at previous timestep
    prev = [( q[i], q_vals[i,n-1]) for i in range(3*nlinks)] +\
           [(dq[i],dq_vals[i,n-1]) for i in range(3*nlinks)]

    r_sub.append(r_sub_params.subs(prev))

    # torque values
    torques = [(tau[0,j],tau_vals[j][n-1]) for j in range(3)] +\
              [(tau[i,j],0) for i in range(1,nlinks) for j in range(3)]

    # substitute previous conditions into EOM
    EOM_sub = EOM_sub_params.subs(prev + torques)
    
    # solve for the acceleration
    accel = np.array([sym.solve(EOM_sub, ddq)[ddq_var] for ddq_var in ddq], dtype=float)
    
    # integrate for the next velocity and next position
    dq_vals[:,n] = dq_vals[:,n-1] + h*accel
    q_vals[ :,n] =  q_vals[:,n-1] + h*dq_vals[:,n]
    
    print('\r%s' %n ,end='') # <- so you know how far into the simulation you are
# current_time = log_time(start_time, "\rSimulation Complete")

prev = [( q[i], q_vals[i,N]) for i in range(3*nlinks)] +\
       [(dq[i],dq_vals[i,N]) for i in range(3*nlinks)]

r_sub.append(r_sub_params.subs(prev))

In [ ]:
from scipy.optimize import fsolve

N = 100
h = 0.01
t = np.arange(0,N*h,h)

#parameters
X0val, Y0val, Z0val,  = 0, 0, 2

mass    = 1.0
length  = 0.5
dCOM    = 0.5

I_phi_val = mass*length**2/12
I_th_val  = mass*length**2/12

r = 0.01 # an example value of the width of the link
I_psi_val = mass*r**2/2

tau_vals = [3*np.sin(8*np.pi*t), 3*np.cos(8*np.pi*t), 0*t]

sym_list1 = [g,
             X0,Y0,Z0]+\
            [m[i] for i in range(nlinks)]+\
            [l[i] for i in range(nlinks)]+\
            [d[i] for i in range(nlinks)]+\
            [I_phi[i] for i in range(nlinks)]+\
            [I_th[i]  for i in range(nlinks)]+\
            [I_psi[i] for i in range(nlinks)]+\
            list(q)+list(dq)

sym_list2 = list(ddq)+\
            [tau[i] for i in range(3*nlinks)]
            

# print(sym_list)

sub_list = [9.81,
            X0val,Y0val,Z0val]+\
           [mass for i in range(nlinks)]+\
           [length for i in range(nlinks)]+\
           [dCOM for i in range(nlinks)]+\
           [I_phi_val for i in range(nlinks)]+\
           [I_th_val for i in range(nlinks)]+\
           [I_psi_val for i in range(nlinks)]

func_map = {'sin':np.sin, 'cos':np.cos}

lamb_EOM = {}
lamb_rs  = {}
for dof_i, dof in enumerate(q):
    dof = str(dof).replace('\\','').replace('_','')
    lamb_EOM[dof] = sym.lambdify(sym_list1+sym_list2, EOM[dof_i] ,modules = [func_map])

for i in range(nlinks+1):
    for j in range(3):
        lamb_rs [3*i + j] = sym.lambdify(sym_list1, rEnds_0[i][j],modules = [func_map])
    
# initial conditions
q_vals = np.zeros((3*nlinks, N+1))
dq_vals = np.zeros((3*nlinks, N+1))

r_sub = []

def solve_ddq(prevs, taus, EOMs):
    def EOM_ddq(ddqs):
        equations = []
        for dof in EOMs:
            equations.append(EOMs[dof](*sub_list, *prevs, *ddqs, *taus))
        return equations

    ddq_sol = fsolve(EOM_ddq, np.zeros(ddq.shape))  # Initial guess for ddq
    return ddq_sol

for n in range(1,N+1):
    # conditions at previous timestep
    X_prev = [ q_vals[i,n-1] for i in range(len(q))] +\
             [dq_vals[i,n-1] for i in range(len(q))]

    
    r_sub.append([lamb_rs[i](*sub_list, *X_prev) for i in range(3*(nlinks+1))])

    # torque values
    tau_prev = [tau_vals[i][n-1] for i in range(3)] +\
               [0 for i in range(3,3*nlinks)]
    
    # solve for the acceleration
    accel = solve_ddq(X_prev,tau_prev,lamb_EOM)
    
    # integrate for the next velocity and next position
    dq_vals[:,n] = dq_vals[:,n-1] + h*accel
    q_vals[ :,n] =  q_vals[:,n-1] + h*dq_vals[:,n]
    
    print('\r%s' %n ,end='')
    
X_prev = [ q_vals[i,N] for i in range(len(q))] +\
         [dq_vals[i,N] for i in range(len(q))]
r_sub.append([lamb_rs[i](*sub_list, *X_prev) for i in range(3*(nlinks+1))])

In [ ]:
#animate it
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib 

fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=[8, 8], dpi=100)

def plot_pendulum(n): #update function for animation
    ax.clear()
    ax.set_xlim([-nlinks*length, nlinks*length])
    ax.set_ylim([-nlinks*length, nlinks*length])
    ax.set_zlim([0, 2*nlinks*length])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    ax.view_init(90,0)  # elevation then azimuth
    
    cols = ['xkcd:black',
            'xkcd:green',
            'xkcd:bright purple',
            'xkcd:teal',
            'xkcd:bright pink',
            'xkcd:bright blue',
            'xkcd:red',
            'xkcd:blue',
            'xkcd:orange']

    for i in range(nlinks):
        top    = r_sub[n][3*i    :3*(i+1)]
        bottom = r_sub[n][3*(i+1):3*(i+2)]

        ax.plot([top[0], bottom[0]],
                [top[1], bottom[1]],
                [top[2], bottom[2]],
                linewidth=2,color = cols[i%9],zorder=100)

update = lambda i: plot_pendulum(i) #lambdify update function

animate = ani.FuncAnimation(fig,update,range(1,N+1),interval = 50,repeat=True) # interval = frame time. 1/50 = 20 fps
# animate = ani.FuncAnimation(fig,update,range(1,N+1),interval = 1000*hm,repeat=True) # if you want it to play at the actual speed

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
ani.Animation.save(animate,'symsolvetest.mp4', fps=int(1/h), dpi=300) # if you want to save the animation instead of embedding it

## **Spherical Pendulum**

## **Hinge Pendulum**